<a href="https://colab.research.google.com/github/aishwaryasharmaccoew/CIS6930-NLP/blob/aishwarya/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSA**

**Loading Data**

In [ ]:
import pandas as pd
df1 = pd.read_csv("/content/NLP_data/depression_post_features_tfidf_256.csv")
df= df1[['post']].copy()
df.head()

,post
0,My heart aches but i cant break Recently my re...
1,Cheers I’m not sure if this is the right sub t...
2,Fuck titles. I’m depressed I feel so fucking w...
3,suffering from low energy I'm not exaggerating...
4,I can already tell This year is just going to ...


**Data Preprocessing** <br>
Steps involved are :
1. Remove all the special characters from the text.
2. Remove all the words with less than 3 letters.
3. Lowercase all the characters.
4. Remove stop words.

In [ ]:
#remove special characters
df['preprocessed_posts'] = df['post'].str.replace("[^a-zA-Z#]", " ")
#remove words have letters less than 3
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
#lowercase all characters
df['preprocessed_posts'] = df['preprocessed_posts'].fillna('').apply(lambda x: x.lower())
#remove stop words

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# tokenization
tokenized_doc = df['preprocessed_posts'].fillna('').apply(lambda x: x.split())
# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
df['preprocessed_posts'] = detokenized_doc
df['preprocessed_posts']

<ipython-input-23-523bab568e00>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preprocessed_posts'] = df['post'].str.replace("[^a-zA-Z#]", " ")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        heart aches cant break recently relationship e...
1        cheers sure right sub post evening soon opened...
2        fuck titles depressed feel fucking worthless a...
3        suffering low energy exaggerating say taking c...
4        already tell year going suck ass worse last ye...
                               ...                        
38028    someone please help take anymore want disappea...
38029    really depressed talked school nurse times cor...
38030    feel alone feel like shit people dont care thi...
38031    messed big want make things right girlfriend e...
38032    suicide attempt survivor guys know depression ...
Name: preprocessed_posts, Length: 38033, dtype: object

**Document Term Matrix**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
dtMatrix = vectorizer.fit_transform(df['preprocessed_posts'])
dtMatrix.get_shape()

(38033, 40530)

**Applying LSA**

In [ ]:
# SVD represent documents and terms in vectors 
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(dtMatrix)

# Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["posts"] = df['preprocessed_posts']
display(topic_encoded_df[["posts", "topic_1", "topic_2"]])

# Features or words used as features 
dictionary = vectorizer.get_feature_names_out()

# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T

,posts,topic_1,topic_2
0,heart aches cant break recently relationship e...,0.2309806433931381,-0.0451195675077753
1,cheers sure right sub post evening soon opened...,0.1003752553958088,-0.0379164654413944
2,fuck titles depressed feel fucking worthless a...,0.1852792754628234,0.0281741285871794
3,suffering low energy exaggerating say taking c...,0.2372180114753215,0.1079603848093750
4,already tell year going suck ass worse last ye...,0.1336300430952184,-0.0184225634586203
...,...,...,...
38028,someone please help take anymore want disappea...,0.1772170210268793,0.0522970880649217
38029,really depressed talked school nurse times cor...,0.2415290464040728,-0.0558942346457298
38030,feel alone feel like shit people dont care thi...,0.2813408267103144,0.2613894529534259
38031,messed big want make things right girlfriend e...,0.2772080384249981,-0.0507287180178646
